
# Der BB84 Quanten-Kryptographie-Algorithmus
Quanten-Schlüsselaustausch mit Qiskit
Stellen wir uns vor, dass zwei Personen, Alice und Bob, über ein unsicheres Medium wie das Internet kommunizieren möchten. Alice möchte eine geheime Nachricht an Bob senden. Da das Internet öffentlich zugänglich ist und jeder die Kommunikation abfangen könnte, muss ein spezielles Verschlüsselungsverfahren entwickelt werden, damit nur Bob mit einem speziellen "Schlüssel" Zugriff auf Alices Nachricht hat.

In diesem Kontext betrachten wir den BB84-Algorithmus. Dies ist ein symmetrisches Verfahren zur Schlüsselaustauschverteilung, das Alice ermöglicht, ihre Nachricht sicher an Bob zu übermitteln. Der BB84-Algorithmus nutzt die Prinzipien der Quantenmechanik, um eine sichere Schlüsselverteilung zu gewährleisten.

Notwendige Bibliotheken importieren
Um den Algorithmus zu implementieren, verwenden wir Qiskit, eine Open-Source-Softwareentwicklung für Quantencomputing. Zu Beginn müssen wir die benötigten Bibliotheken importieren:

In [201]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np
%matplotlib inline

### Verschlüsselung mithilfe von Quantenkryptographie

![credit:norwegiancreations.com](https://raw.githubusercontent.com/jnwee/quantum-keydistribution-studproj/refs/heads/main/bb84.png?token=GHSAT0AAAAAACZQACJD5VXPTNROQFWHDZBSZ2ZPRFQ)

#### Quanten- und klassische Kanäle
- **Klassischer Kanal:** Ein klassischer Kommunikationskanal könnte eine Telefonleitung oder ein Netzwerkdraht sein, bei dem elektrische Signale Bits oder codierte Informationen darstellen. 
- **Quantenkanal:** Ein Quantenkommunikationskanal wird häufig durch Glasfaserkabel realisiert, da Quantenkryptographie über Polarisationszustände von Photonen implementiert wird. Diese Zustände repräsentieren Qubits.

---

### Schritt 1: Generieren von Binärstrings

Alice erzeugt zwei zufällige Binärstrings:
1. **Zustands-String:** Dieser String enthält die Zustände der Qubits (0 oder 1).
2. **Basis-String:** Dieser String gibt an, welche Basis (computational oder Hadamard) zur Kodierung der Qubits verwendet wird.

#### Darstellungen in den Basen
- **Hadamard-Basis:**  
  $$0\ wird\ als\ \\frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)\ dargestellt$$
  $$1\ wird\ als\ \\frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)\ dargestellt$$
  - Dies entspricht der Anwendung eines Hadamard-Tores auf die computational Basis $$(|0\rangle) \ oder (|1\rangle)$$
- **Computational-Basis:**  
  - Bits werden klassisch dargestellt: $$(|0\rangle)\ für\ 0\ und\ (|1\rangle) \ für\ 1$$

  Hier ist die korrigierte Version für korrekte LaTeX-Darstellungen:

---

Die Wahl der Basis ist entscheidend, da Bob dieselbe Basis verwenden muss, um die Nachricht korrekt zu entschlüsseln. Nur Bits mit übereinstimmenden Basen werden für die Schlüsselbildung genutzt.

---

### Beispiel: Alice erstellt zwei Strings
Alice generiert:
1. Einen **Zustands-String**, der die Qubit-Zustände (0 oder 1) beschreibt.  
2. Einen **Basis-String**, der die Kodierungsbasis für jedes Qubit angibt (0 = computational, 1 = Hadamard).  

Die Informationen über die verwendete Basis sind entscheidend, damit Bob die Nachricht entschlüsseln kann. Im nächsten Schritt werden die kodierten Qubits über den Quantenkanal gesendet.

In [202]:
np.random.seed(42)
n = 24  # Number of qubits
alice_bits = np.random.randint(2, size=n)
alice_basis = np.random.randint(2, size=n)
bob_basis = np.random.randint(2, size=n)

key_agreement = [ab == bb for ab, bb in zip(alice_basis, bob_basis)]
print('Alices Bits:   ', alice_bits)
print('Alices Basis:  ', alice_basis)
print('Bobs Basis:    ', bob_basis)

Alices Bits:    [0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 0 1 0 1 1]
Alices Basis:   [1 1 1 1 1 1 0 0 1 1 1 0 1 0 0 0 0 0 1 1 1 1 1 0]
Bobs Basis:     [1 1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1]


### Übersetzung: BB84-Protokoll – Kodierung und Schaltungserstellung

#### Kodierung im BB84-Protokoll
- Wenn Alices Zustand `1` ist und ihre Basis `0`, bedeutet dies, dass sie die **computational Basis** verwendet, um den Zustand `1` zu kodieren. 
- Wenn Alices Zustand `1` ist und ihre Basis `1`, bedeutet dies, dass sie das **Hadamard-Tor** verwendet, um den Zustand `1` zu kodieren. Dadurch wird der Zustand in die Superposition versetzt.

---

### Schritt 2: Erstellen der Schaltung mithilfe des BB84-Protokolls

Dieses Protokoll wird verwendet, um eine Quanten-Schaltung zu erstellen:

- **Kodierung durch Alice:**
  1. Wenn Alice ein `1` in ein Qubit kodieren möchte, wendet sie ein **Pauli-X-Gatter** auf das Qubit an. Das Pauli-X-Gatter ist äquivalent zu einem **NOT-Gatter**, das den Zustand von `0` auf `1` ändert.
  2. Wenn Alice ein `0` kodieren möchte, ist keine Aktion erforderlich, da die Qubits in Qiskit standardmäßig auf den Zustand `0` initialisiert sind.
  3. Für die Kodierung in der **Hadamard-Basis** wendet Alice ein **Hadamard-Tor** auf das Qubit an. Dies versetzt das Qubit in eine Superposition.
  4. Für die Kodierung in der **computational Basis** ist keine weitere Aktion notwendig, da die Standardbasis in Qiskit die computational Basis ist.

- **Senden der Qubits:**  
  Alice sendet die kodierten Qubits über den Quantenkanal an Bob.

---

### Messung durch Bob
- Bob misst die empfangenen Qubits entsprechend seiner eigenen Binärzeichenkette (Basiswahl):
  1. Um ein Qubit in der **Hadamard-Basis** zu messen, wendet Bob zunächst ein **Hadamard-Tor** auf das entsprechende Qubit an. Dadurch wird die Messung in der computational Basis ermöglicht.
  2. Anschließend führt Bob die Messung in der **computational Basis** durch, um das Ergebnis zu erhalten.

Bob kann anhand der Ergebnisse und durch Abgleich mit Alice die korrekte Schlüsselgenerierung überprüfen.

In [203]:
def bb84_circuit(state, basis, measurement_basis):
   
    #state: array of 0s and 1s denoting the state to be encoded
    #basis: array of 0s and 1s denoting the basis to be used for encoding
                #0 -> Computational Basis
                #1 -> Hadamard Basis
    #meas_basis: array of 0s and 1s denoting the basis to be used for measurement
                #0 -> Computational Basis
                #1 -> Hadamard Basis
    
    num_qubits = len(state)
    
    bb84_circuit = QuantumCircuit(num_qubits)

    # Sender prepares qubits
    for i in range(len(basis)):
        if state[i] == 1:
            bb84_circuit.x(i)
        if basis[i] == 1:
            bb84_circuit.h(i)
   

    # Measuring action performed by Bob
    for i in range(len(measurement_basis)):
        if measurement_basis[i] == 1:
            bb84_circuit.h(i)

       
    bb84_circuit.measure_all()
    
    return bb84_circuit

### Schritt 3: Erstellung des Schlüssels

Alice und Bob behalten nur die Bits in ihren Zeichenketten, bei denen die verwendeten Basen übereinstimmen. Zum Beispiel werden Bits nur dann beibehalten, wenn:
- Alices Kodierungsbasis `0` ist und Bobs Messbasis ebenfalls `0`, oder
- Alices Kodierungsbasis `1` ist und Bobs Messbasis ebenfalls `1`.

#### Wahrscheinlichkeit, dass die Basen übereinstimmen
Die Wahrscheinlichkeit, dass Alices und Bobs Basen übereinstimmen, beträgt:  
- Die Wahrscheinlichkeit, dass Alice die **computational Basis** wählt, ist $$\frac{1}{2}$$
- Die Wahrscheinlichkeit, dass Bob die **computational Basis** wählt, ist ebenfalls $$\frac{1}{2}$$
Daher ist die Wahrscheinlichkeit, dass beide die computational Basis wählen, $$\frac{1}{2} \times \frac{1}{2} = \frac{1}{4}$$
Analog dazu ist die Wahrscheinlichkeit, dass beide die **Hadamard-Basis** wählen, ebenfalls $$\frac{1}{4}$$

Insgesamt ist die Wahrscheinlichkeit, dass die Basen übereinstimmen:  
$$ P = \frac{1}{4} + \frac{1}{4} = \frac{1}{2} $$
Das bedeutet, dass Alice und Bob in etwa der Hälfte der Fälle übereinstimmende Basen haben werden.

---

### Schritt 4: Ausführen der Schaltung

Alice und Bob vergleichen ihre Basen und behalten nur die Bits, bei denen die Basen übereinstimmen:
- Wenn `alice_basis[i] == bob_basis[i]`, wird das entsprechende Bit beibehalten.
- Andernfalls wird das Bit verworfen.

Dies stellt sicher, dass der endgültige Schlüssel nur aus den Bits besteht, bei denen Alice und Bob sicher sind, dass sie die gleiche Basis verwendet haben.

In [204]:
circuit = bb84_circuit(alice_bits, alice_basis, bob_basis)
simulator = AerSimulator()
compiled_circuit = transpile(circuit, simulator)
sim_result = simulator.run(compiled_circuit, shots=1).result()
counts = sim_result.get_counts()
key = max(counts, key=counts.get)

encryption_key = ''
for i in range(len(alice_bits)):
    if alice_basis[i] == bob_basis[i]:
         encryption_key += str(key[i])
print(f"Key: {encryption_key}")

Key: 111100101001001


Das ist der Schlüssel, den Bob verwenden kann, um eine Nachricht von Alice zu entschlüsseln.